In [2]:
## Import required packages

!pip install pandas
!pip install psycopg2
import pandas as pd

In [3]:
## Task 1A(1)
csv_file = "/Users/temit/Desktop/My downloads/freshmart_products_cstone.csv"   #extract downloaded data from local machine

In [5]:
productsid_df = pd.read_csv(csv_file)  ## load the extracted data into a pandas dataframe

In [33]:
## Task 1A(2)
# productsid_df  ## print loaded data

productsid_df.head(5)  ## dispay the first five rows - ascending order by default

,ProductID,ProductName,Category,Price,StockQuantity,StockValue
0,1,Milk,Dairy,19.35,358.0,6927.30
1,2,Cola,Beverages,9.99,81.0,809.19
2,3,Banana,Produce,30.33,112.0,3396.96
3,4,Cream,Dairy,22.85,224.0,5118.40
4,5,Cream,Dairy,3.85,97.0,373.45


In [8]:
## Task 1A(2) clean data by removing int from str in 'ProductName's table' which converts it to a regex

productsid_df["ProductName"] = productsid_df["ProductName"].str.replace(r"\d+", "", regex=True).str.strip() # used .str.replace(r"\d+ to remove int, 
#"" to eplace the space with nothing, regex=True to conert text to regex and .str.strip to remove any whitespaces")

In [12]:
## Task 1A(2) continue data cleaning by no-nulls and it's total 

productsid_df.isnull().sum()
# productsid_df.isnull()  ## confirm no-null cleaning

ProductID        0
ProductName      0
Category         0
Price            0
StockQuantity    5
dtype: int64

In [11]:
## Task 1A(2) continues..

productsid_df["Price"] = productsid_df["Price"].fillna(0)  ## used .fillna(0) to replace no-nulls with 0 in "Price" column

In [13]:
## Task 1A(2) continues..

productsid_df["StockQuantity"] = productsid_df["StockQuantity"].fillna(0)   ## used .fillna(0) to replace no-nulls with 0 in "StockQuantity" column

In [14]:
## Task 1A(2) continues..

productsid_df.isnull().sum

<bound method DataFrame.sum of      ProductID  ProductName  Category  Price  StockQuantity
0        False        False     False  False          False
1        False        False     False  False          False
2        False        False     False  False          False
3        False        False     False  False          False
4        False        False     False  False          False
..         ...          ...       ...    ...            ...
200      False        False     False  False          False
201      False        False     False  False          False
202      False        False     False  False          False
203      False        False     False  False          False
204      False        False     False  False          False

[205 rows x 5 columns]>

In [18]:
## Task 1A(2) continues..

productsid_df.duplicated()  # used .duplicated() to check for duplicates

0      False
1      False
2      False
3      False
4      False
       ...  
200     True
201     True
202    False
203     True
204     True
Length: 205, dtype: bool

In [20]:
## Task 1A(2) continues..

productsid_df = productsid_df.drop_duplicates()   # used .drop_duplicates() to remove duplicated data

In [23]:
## Task 1A(2) continues..

productsid_df.drop_duplicates()  # assigned updated data to the dataset

,ProductID,ProductName,Category,Price,StockQuantity
0,1,Milk,Dairy,19.35,358.0
1,2,Cola,Beverages,9.99,81.0
2,3,Banana,Produce,30.33,112.0
3,4,Cream,Dairy,22.85,224.0
4,5,Cream,Dairy,3.85,97.0
...,...,...,...,...,...
196,197,Cola,Beverages,27.04,56.0
197,198,Apple,Produce,35.28,433.0
198,199,Mineral Water,Beverages,36.58,278.0
199,200,Donut,Bakery,48.51,115.0


In [26]:
## Task 1A(2) continues..

productsid_df = productsid_df.drop_duplicates()  # assigned updated data to the dataset
# productsid_df.duplicated().sum()  # confirmed there's no more duplicate

In [29]:
productsid_df["Category"] = productsid_df["Category"].replace("snakcs", "snacks")   # used .replace() to correct wrongly spelt word 'snakc'on row 30 and assigned updated data to Category
#productsid_df.iloc[30]  # reprint row 30 to confirm the update

In [ ]:
## Task 1A(2) continues..

productsid_df.sort_values(by = 'Category', ascending=False)
# productsid_df['Category'].info()  # .info() was used to confirm sorting by Category

In [37]:
## Task 1A(2) continues..

productsid_df["StockValue"] = productsid_df["Price"] * productsid_df["StockQuantity"]
# productsid_df  ## run to confirm the new column "StockQuantity"

In [39]:
## Task 1A(2) continues..
# Group by Category and aggregate
agg_productsid_df = productsid_df.groupby("Category").agg(Avg_Price=("Price", "mean"),
    Total_StockQuantity=("StockQuantity", "sum")).reset_index()

agg_productsid_df

,Category,Avg_Price,Total_StockQuantity
0,Bakery,21.815135,9898.0
1,Bevarages,9.360000,176.0
2,Beverages,24.810488,11304.0
3,Dairy,24.242195,9649.0
4,Produce,24.264878,9580.0
5,Snacks,25.313947,10673.0
6,dairy,9.990000,469.0
7,snacks,21.830000,63.0


In [152]:
import psycopg2
import pandas as pd

# -----------------------------
# Step 1: Load CSV into Pandas
# -----------------------------
csv_path = r"C:\Users\temit\Desktop\My downloads\freshmart_products_cstone.csv"
productsid_df = pd.read_csv(csv_path)

productsid_df.head()

,ProductID,ProductName,Category,Price,StockQuantity
0,1,Milk 1,Dairy,19.35,358.0
1,2,Cola 2,Beverages,9.99,81.0
2,3,Banana 3,Produce,30.33,112.0
3,4,Cream 4,Dairy,22.85,224.0
4,5,Cream 5,Dairy,3.85,97.0


In [154]:
# -----------------------------
# Step 2: Connect to PostgreSQL
# -----------------------------
try:
    conn = psycopg2.connect(
        host="localhost",
        port="5432",
        dbname="freshmart_db",
        user="postgres",
        password="OluwaBori#77"
    )
    cur = conn.cursor()
    
except Exception as e:
#    print("❌ Connection failed:", e)
    exit()

In [155]:
productsid_df['StockQuantity'] = pd.to_numeric(productsid_df['StockQuantity'], errors='coerce').fillna(0).astype(int)

In [156]:
productsid_df['Price'] = pd.to_numeric(productsid_df['Price'], errors='coerce').fillna(0).astype(float)

In [160]:
# -----------------------------
# Step 3: Create table if not exists
# -----------------------------
create_table_query = """
CREATE TABLE IF NOT EXISTS Products (
    ProductID SERIAL PRIMARY KEY,
    ProductName VARCHAR(100),
    Category VARCHAR(50),
    Price DECIMAL(10,2),
    StockQuantity INT  
);
"""
cur.execute(create_table_query)
conn.commit()

In [161]:
#cur.execute('''
#DROP TABLE IF EXISTS Products
#''')

In [162]:
# -----------------------------
# Step 4: Insert Data from Pandas
# -----------------------------
try:
    for _, row in productsid_df.iterrows():
        cur.execute("""
            INSERT INTO Products (ProductName, Category, Price, StockQuantity)
            VALUES (%s, %s, %s, %s)
        """, (row['ProductName'], row['Category'], row['Price'], row['StockQuantity']))

    conn.commit()
   # print("✅ All data inserted successfully!")

except Exception as e:
    conn.rollback()  # reset transaction if error happens

In [163]:
# -----------------------------
# Step 5: Verify
# -----------------------------
# Run a query
cur.execute("""
    SELECT * 
    FROM Products 
    LIMIT 5;
""")

# Fetch results
for row in cur.fetchall():
    print(row)

# Close connections
cur.close()
conn.close()

(1, 'Milk 1', 'Dairy', Decimal('19.35'), 358)
(2, 'Cola 2', 'Beverages', Decimal('9.99'), 81)
(3, 'Banana 3', 'Produce', Decimal('30.33'), 112)
(4, 'Cream 4', 'Dairy', Decimal('22.85'), 224)
(5, 'Cream 5', 'Dairy', Decimal('3.85'), 97)


In [164]:
import psycopg2

# First connect to default database (postgres)
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    dbname="postgres",   # default db
    user="postgres",
    password="OluwaBori#77"
)
conn.autocommit = True
cur = conn.cursor()

# Create freshmart_db if it doesn't exist
cur.execute("SELECT 1 FROM pg_database WHERE datname='freshmart_db'")
exists = cur.fetchone()
if not exists:
    cur.execute("CREATE DATABASE freshmart_db;")
    print("✅ Database freshmart_db created.")
else:
    print("ℹ️ Database already exists.")

cur.close()
conn.close()

ℹ️ Database already exists.


In [165]:
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    dbname="freshmart_db",
    user="postgres",
    password="OluwaBori#77"
)
cur = conn.cursor()

In [166]:
# Data Definition Language (DDL)
# Create tables

create_table_query = """
CREATE TABLE IF NOT EXISTS Products (
    ProductID SERIAL PRIMARY KEY,
    ProductName VARCHAR(100),
    Category VARCHAR(50),
    Price DECIMAL,
    StockQuantity INT
);
"""
cur.execute(create_table_query)
conn.commit()

In [167]:
conn.rollback()

In [168]:
# Insert data into PostgreSQL
for _, row in productsid_df.iterrows():
    cur.execute("""
        INSERT INTO Products (ProductName, Category, Price, StockQuantity)
        VALUES (%s, %s, %s, %s)
    """, (row['ProductName'], row['Category'], row['Price'], row['StockQuantity']))

conn.commit()

✅ Data inserted into Products table


In [ ]:
git init
git add .
git commit -m "Initial FreshMart data pipeline"
git remote add origin https://github.com/yourusername/freshmart-pipeline.git
git push -u origin main